In [ ]:
# Import Libraries

from sbcbinaryformat import Streamer, Writer
import numpy as np
import matplotlib.pyplot as plt

# from GetEvent import GetEvent, GetScint
import GetEvent
# from ana.SiPMPulses import SiPMPulses
from ana import SiPMPulses
import importlib

from scipy.optimize import curve_fit
import os

In [ ]:
1+1

In [ ]:
importlib.reload(GetEvent)
importlib.reload(SiPMPulses)

In [ ]:
# matplotlib parameters and output locations
plt.rcParams.update({'font.size': 14})

dosave = False
savedir = "./figures/"
os.makedirs(savedir, exist_ok=True)

In [ ]:
FILES = [
    "/exp/e961/data/SBC-25-daqdata/20251008_1", # Background 10/8, GAr, 54V bias, 1950 trig. thresh
    "/exp/e961/data/SBC-25-daqdata/20251015_22", # Background 10/15, LAr, 54V bias, 1950 trig. thresh
    "/exp/e961/data/SBC-25-daqdata/20251020_11", # Background, 10/20, LAr, 54V bias, 1950 trig. thresh
    "/exp/e961/data/SBC-25-daqdata/20251021_2", # Background, 10/21, LAr, 54V bias, 1800 trig. thresh
    "/exp/e961/data/SBC-25-daqdata/20251024_4.tar", # Background, 10/24, LAr, 54V bias, 1800 trig. thresh
    "/exp/e961/data/SBC-25-daqdata/20251024_5.tar", # Background, 10/24, LAr, 51V bias, 1800 trig. thresh
    "/exp/e961/data/SBC-25-daqdata/20251024_7.tar", # Background, 10/24, LAr, 51V bias, 1900 trig. thresh
    "/exp/e961/data/SBC-25-daqdata/20251027_0.tar", # Background, 10/27, LAr, 51V bias, 1900 trig. thresh
    "/exp/e961/data/SBC-25-daqdata/20251027_1.tar", # Background, 10/27, LAr, 54V bias, 1900 trig. thresh
]
DATE = "10-8-27"
labels = [
    "10/8 GAr, 54V bias, 1950 trig.",
    "10/15 LAr, 54V bias, 1950 trig.",
    "10/20 LAr, 54V bias, 1950 trig.",
    "10/21 LAr, 54V bias, 1800 trig.",
    "10/24 LAr, 54V bias, 1800 trig.",
    "10/24 LAr, 51V bias, 1800 trig.",
    "10/24 LAr, 51V bias, 1900 trig.",
    "10/27 LAr, 51V bias, 1900 trig.",
    "10/27 LAr, 54V bias, 1900 trig.",
]

In [ ]:
RUNS = [F.split("/")[-1].rstrip(".tar") for F in FILES]
TITLES = ["Run %s, %s" % (R, L) for (R, L) in zip(RUNS, labels)]

In [ ]:
RUNS

In [ ]:
GetEvent.NEvent(FILES[0])

In [ ]:
data = {}

for F in FILES:
    print(F)
    data[F] = []
    for e in range(GetEvent.NEvent(F)):
        data[F].append(GetEvent.GetEvent(F, e, strictMode=False))

In [ ]:
# Get the first 500 waveforms
wvfs = {}

for F in FILES:
    wvfs[F] = data[F][0]["scintillation"]["Waveforms"](end=1000)

In [ ]:
times = np.arange(wvfs[FILES[0]].shape[-1])/(62.5)

In [ ]:
times.shape

In [ ]:
SAMPLE_RATE = (data[FILES[0]][0]["scintillation"]["sample_rate"] / 1e6)

SAMPLE_RATE

In [ ]:
is_bad_event = {}

for F in FILES:
    avg_wvf = np.sum(wvfs[F], axis=1)
    
    avg_fft = np.abs(np.fft.rfft(avg_wvf, axis=1))[:, 1:]
    fft_freqs = np.fft.rfftfreq(avg_wvf.shape[1], d=1/SAMPLE_RATE)[1:]
    
    is_bad_event[F] = fft_freqs[np.argmax(avg_fft, axis=1)] > 5 # MHz
    print(F, is_bad_event[F].mean())

In [ ]:
ifig = 0
for i, (F, run, title) in enumerate(zip(FILES, RUNS, TITLES)):
    for ievt in range(min(5, wvfs[F].shape[0])):
        plt.figure(ifig)
        if not np.any(is_bad_event[F]): continue
                
        # plot the first triggered waveform in each channel
        for channel in range(wvfs[F].shape[1]):
            plt.plot(times, wvfs[F][ievt, channel, :])
        
        plt.title(title)
        plt.xlabel("Time [$\\mu$s]")
        plt.ylabel("Amplitude [ADC]")

        ifig += 1
        if dosave:
            plt.savefig(savedir + "ex_waveforms_evt%i_R%s.pdf" % (ievt, run), bbox_inches="tight")

In [ ]:
for i, (F, run, title) in enumerate(zip(FILES, RUNS, TITLES)):
    plt.figure(i)
    if not np.any(is_bad_event[F]): continue
        
    INDEX = np.where(is_bad_event[F])[0][0]
    
    # plot the first triggered waveform in each channel
    for channel in range(wvfs[F].shape[1]):
        plt.plot(times, wvfs[F][INDEX, channel, :])
    
    plt.title(title)
    plt.xlabel("Time [$\\mu$s]")
    plt.ylabel("Amplitude [ADC]")
    
    if dosave:
        plt.savefig(savedir + "ex_waveforms_bad_R%s.pdf" % run, bbox_inches="tight")

In [ ]:
for i, (F, run, title) in enumerate(zip(FILES, RUNS, TITLES)):
    plt.figure(i)
    INDEX = np.where(~is_bad_event[F])[0][0]
    
    # plot the first triggered waveform in each channel
    for channel in range(wvfs[F].shape[1]):
        plt.plot(times, wvfs[F][INDEX, channel, :])
    
    plt.title(title)
    plt.xlabel("Time [$\\mu$s]")
    plt.ylabel("Amplitude [ADC]")
    
    if dosave:
        plt.savefig(savedir + "ex_waveforms_R%s.pdf" % run, bbox_inches="tight")

In [ ]:
importlib.reload(SiPMPulses)

In [ ]:
# Run the SiPM analysis
sipm_out = {}
for F in FILES:
    for e in range(min(GetEvent.NEvent(F), 3)):
        this_sipm_out = SiPMPulses.SiPMPulsesBatched(data[F][e], n_sigma_threshold=6, smoothing=7, 
                                                       maxwvf=1_000_000, progress=True, njob=3)
        if e == 0:
            sipm_out[F] = this_sipm_out
        else:
            for k in sipm_out[F].keys():
                sipm_out[F][k] = np.concatenate((sipm_out[F][k], this_sipm_out[k]), axis=1)

In [ ]:
def smoothed(trace, smoothing):
    trace_cumsum = np.cumsum(trace, axis=0)
    return (trace_cumsum[smoothing:] - trace_cumsum[:-smoothing]) / smoothing

In [ ]:
ifig = 0
for F, R, T in zip(FILES, RUNS, TITLES):
    EVENT = np.where(~is_bad_event[F])[0][0]
    
    this_sipm = sipm_out[F]
    whichchan = np.where(~np.isnan(this_sipm["hit_t0"][:, EVENT]))[0]
    for chan in whichchan[:3]:
        plt.figure(ifig)
        
        plt.plot(times, wvfs[F][EVENT, chan, :])
        SMOOTHING = 7
        plt.plot(times[SMOOTHING//2:-SMOOTHING//2], smoothed(wvfs[F][EVENT, chan, :], SMOOTHING))
    
        hit_times = (times[SMOOTHING//2:-SMOOTHING//2] >= this_sipm["hit_t0"][chan, EVENT]) &\
                    (times[SMOOTHING//2:-SMOOTHING//2] < this_sipm["hit_tf"][chan, EVENT])
        plt.fill_between(times[SMOOTHING//2:-SMOOTHING//2][hit_times], smoothed(wvfs[F][EVENT, chan, :], SMOOTHING)[hit_times], 
                         this_sipm["baseline"][chan, EVENT],
                        color="lightgray")
        plt.axvline([this_sipm["hit_t0"][chan, EVENT]], color="red")
        plt.axvline([this_sipm["hit_tf"][chan, EVENT]], color="red")
        plt.axhline([this_sipm["baseline"][chan, EVENT]], color="red", linestyle="--")
        plt.axhline([this_sipm["baseline"][chan, EVENT] - this_sipm["hit_amp"][chan, EVENT]], color="red", linestyle="--")
        
        plt.title("%s. Channel: %i" % (T, chan))
    
        
        plt.xlabel("Time [$\\mu$s]")
        plt.ylabel("Amplitude [ADC]")

        ifig += 1
        if dosave:
            plt.savefig(savedir + "ex_wvfana_C%i_R%s.pdf" % (chan, R), bbox_inches="tight")

In [ ]:
hit_width = {}
is_bad_event = {}

for F in FILES:
    this_sipm = sipm_out[F]

    hit_width[F] = this_sipm["hit_tf"] - this_sipm["hit_t0"]
    is_bad_event[F] = this_sipm["max_avg_fft_freq"] > 5 # MHz

In [ ]:
ifig = 0

for F, T, R in zip(FILES, TITLES, RUNS):
    plt.figure(ifig)
    _ = plt.hist(hit_width[F].flatten()[~is_bad_event[F].flatten()], bins=np.linspace(0, 1, 21))
    
    plt.title(T)
    plt.xlabel("Hit Width [$\\mu$s]")
    plt.ylabel("# Hits")
    
    plt.axvline([0.2], color="green", linewidth=3)

    ifig += 1
    if dosave:
        plt.savefig(savedir + "hit_widths_R%s.pdf" % R, bbox_inches="tight")

In [ ]:
good_hit = {}
for F in FILES:
    good_hit[F] = (hit_width[F] > 0.2) & (~is_bad_event[F])

In [ ]:
this_sipm["hit_amp"].shape

In [ ]:
def gauss(X, A, mu, sig):
    return A*np.exp(-(X-mu)**2 / (2*sig**2))

In [ ]:
GAIN = {}

for ifig, (F, T, R) in enumerate(zip(FILES, TITLES, RUNS)):
    plt.figure(ifig)
    
    this_sipm = sipm_out[F]
    N,bins,_ = plt.hist(this_sipm["hit_amp"].flatten()[good_hit[F].flatten()], bins=np.linspace(0, 3e2, 151))
    centers = (bins[1:] + bins[:-1])/2
    
    peak1 = centers[np.argmax(N)]
    peak2 = centers[np.argmax(N[centers > 50]) + np.argmax(centers > 50)]
    
    print(peak1, peak2)

    p0 = [np.max(N), peak1, 50]
    whichfit = np.abs(np.arange(N.size) - np.argmax(N)) < 5
    popt, perr = curve_fit(gauss, centers[whichfit], N[whichfit], p0=p0)
    fit_peak1 = popt[1]
    plt.plot(centers, gauss(centers, *popt), color="red", linestyle="--")
    print(fit_peak1)
    
    GAIN[F] = fit_peak1
    
    plt.title(T)
    plt.xlabel("Hit Amplitude [ADC]")
    plt.ylabel("# Hits")
    
    plt.text(0.95, 0.95, "SPE Peak: %.1f ADC" % fit_peak1,
             horizontalalignment="right", verticalalignment="top", transform=plt.gca().transAxes)
    
    plt.axvline([fit_peak1], color="red")
    plt.axvline([fit_peak1*2], color="red")
    plt.axvline([fit_peak1*3], color="red")
    
    if dosave:
        plt.savefig(savedir + "hit_amp_wfit_R%s.pdf" % R, bbox_inches="tight")

In [ ]:
for ifig, (F, T, R) in enumerate(zip(FILES, TITLES, RUNS)):
    plt.figure(ifig)
    
    this_sipm = sipm_out[F]
    N,bins,_ = plt.hist(this_sipm["hit_amp"].flatten()[good_hit[F].flatten()], bins=np.linspace(0, 3e2, 151))
    centers = (bins[1:] + bins[:-1])/2    
    plt.axvline([GAIN[FILES[0]]], color="red")
    plt.axvline([GAIN[FILES[0]]*2], color="red")
    plt.axvline([GAIN[FILES[0]]*3], color="red")

    plt.title(T)
    plt.xlabel("Hit Amplitude [ADC]")
    plt.ylabel("# Hits")
    
    if dosave:
        plt.savefig(savedir + "hit_amp_R%s.pdf" % R, bbox_inches="tight")

In [ ]:
for ifig, (F, T, R) in enumerate(zip(FILES, TITLES, RUNS)):
    plt.figure(ifig)
    
    this_sipm = sipm_out[F]
    N,bins,_ = plt.hist(this_sipm["hit_area"].flatten()[good_hit[F].flatten()], bins=np.linspace(0, 4e3, 151))
    centers = (bins[1:] + bins[:-1])/2
    
    peak1 = centers[np.argmax(N)]
    peak2 = centers[np.argmax(N[centers > 50]) + np.argmax(centers > 50)]
    
    print(peak1, peak2)

    p0 = [np.max(N), peak1, 50]
    whichfit = np.abs(np.arange(N.size) - np.argmax(N)) < 5
    popt, perr = curve_fit(gauss, centers[whichfit], N[whichfit], p0=p0)
    fit_peak1 = popt[1]
    plt.plot(centers, gauss(centers, *popt), color="red", linestyle="--")
    print(fit_peak1)
        
    plt.title(T)
    plt.xlabel("Hit Area [ADC]")
    plt.ylabel("# Hits")
    
    plt.text(0.95, 0.95, "SPE Peak: %.1f ADC" % fit_peak1,
             horizontalalignment="right", verticalalignment="top", transform=plt.gca().transAxes)
    
    plt.axvline([fit_peak1], color="red")
    plt.axvline([fit_peak1*2], color="red")
    plt.axvline([fit_peak1*3], color="red")
    
    if dosave:
        plt.savefig(savedir + "hit_area_R%s.pdf" % R, bbox_inches="tight")

In [ ]:
hit_pe = {}
total_hit_pe = {}

# NOTE: This uses the 1PE peak in each run to calibrate itself.
#       You may instead want to take this from external pulser data.
for F in FILES:
    hit_pe[F] = sipm_out[F]["hit_amp"]/GAIN[F]
    total_hit_pe[F] = np.nansum(hit_pe[F]*good_hit[F].astype(int), axis=0)

In [ ]:
hit_pe

In [ ]:
for ifig, (F, T, R) in enumerate(zip(FILES, TITLES, RUNS)):
    plt.figure(ifig)
    _ = plt.hist(total_hit_pe[F], bins=np.linspace(0, 100, 11), linewidth=3)
    
    
    plt.title(T)
    plt.xlabel("Summed PE from All SiPMs")
    plt.ylabel("# Events")
    
    if dosave:
        plt.savefig(savedir + "hit_pe_R%s.pdf" % R, bbox_inches="tight")

In [ ]:
for ifig, (F, T, R, L) in enumerate(zip(FILES, TITLES, RUNS, labels)):
    _ = plt.hist(total_hit_pe[F], bins=np.linspace(0, 250, 26), 
                 linewidth=3, histtype="step", density=True, label=L)
    
    
plt.xlabel("Summed PE from All SiPMs")
plt.ylabel("Area Normalized")
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.3),
          ncol=2)
if dosave:
    plt.savefig(savedir + "hit_pe_all_%s.pdf" % DATE, bbox_inches="tight")

In [ ]:
for ifig, (F, T, R, L) in enumerate(zip(FILES, TITLES, RUNS, labels)):
    weights = np.full(total_hit_pe[F].shape, 1e3/sum([d["event_info"]["ev_livetime"] for d in data[F]]))
    _ = plt.hist(total_hit_pe[F], bins=np.linspace(0, 200, 21), weights=weights,
                 linewidth=3, histtype="step", label=L)

plt.xlabel("Summed PE from All SiPMs")
plt.ylabel("Events / second")
# plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.3),
#           ncol=2)

plt.legend()

if dosave:
    plt.savefig(savedir + "hit_pe_all_rate_%s.pdf" % DATE, bbox_inches="tight")

In [ ]:
CHANNEL_TO_SIPM = [11, 12, 14, 15, 22, 23, 25, 
                   31, 32, 34, 35, 42, 45, 46,
                   51, 52, 54, 55, 62, 63, 65, 66,
                   71, 72, 74, 75, 82, 83, 85, 86]

CHANNEL_TO_LEVEL = [1, 2, 4, 5, 4, 3, 5,
                    1, 2, 4, 5, 4, 5, 6,
                    1, 2, 4, 5, 4, 3, 5, 6,
                    1, 2, 4, 5, 4, 3, 5, 6]

CHANNEL_TO_PANEL = [1, 1, 1, 1, 2, 2, 2,
                    3, 3, 3, 3, 4, 4, 4,
                    5, 5, 5, 5, 6, 6, 6, 6,
                    7, 7, 7, 7, 8, 8, 8, 8]
NLEVEL = 6
NPANEL = 8
NCHANNEL = 30

In [ ]:
heatmap_arr = {}

for F in FILES:
    heatmap_arr[F] = np.zeros((NLEVEL*2+1, NPANEL*2+1))
    
    for ch in range(NCHANNEL):
        level = CHANNEL_TO_LEVEL[ch]
        panel = CHANNEL_TO_PANEL[ch]
        heatmap_arr[F][level*2-1, panel*2-1] = np.nansum(hit_pe[F][ch, :])*1e3/sum([d["event_info"]["ev_livetime"][0] for d in data[F]])

In [ ]:
bad_sipms = [63]
bad_chans = [ch for ch in range(len(CHANNEL_TO_SIPM)) if CHANNEL_TO_SIPM[ch] in bad_sipms]

good_chan = {}

for F in FILES:
    good_chan[F] = np.full(good_hit[F].shape, True)
    for ch in bad_chans:
        good_chan[F][ch, :] = False

In [ ]:
for ifig, (F, T, R, L) in enumerate(zip(FILES, TITLES, RUNS, labels)):
    plt.figure(ifig)
    plt.imshow(heatmap_arr[F], origin="lower", extent=[0.25, NPANEL+0.75, 0.25, NLEVEL+0.75])
    plt.xticks(list(range(1, NPANEL+1)))
    plt.yticks(list(range(1, NLEVEL+1)))
    plt.gca().xaxis.grid(True)
    plt.gca().yaxis.grid(True)
    plt.xlabel("Panel")
    plt.ylabel("Level")
    cbar = plt.colorbar()
    cbar.set_label('Hit PE / second', fontsize=12, rotation=-90, labelpad=15)
    plt.title(T)

    if dosave:
        plt.savefig(savedir + "hit_pe_heatmap_R%s.pdf" % R, bbox_inches="tight")